## Pratica 1 - Aprendizaje automatico

En este notebook desarrollo la Práctica 1 de Aprendizaje Automático sobre un conjunto de datos bancario, con el objetivo de construir un flujo completo y reproducible desde el análisis inicial hasta la obtención de un modelo final listo para usarse y desplegarse. El trabajo se organiza en fases: 
- (1) EDA, para comprender la estructura del dataset, la calidad de los datos y el comportamiento de la variable objetivo deposit; 
- (2) preprocesamiento mediante pipelines, justificando decisiones como imputación, escalado y codificación; 
- (3) entrenamiento y comparación de modelos (básicos y avanzados) usando validación cruzada interna para seleccionar hiperparámetros; 
- (4) evaluación final con una partición holdout para estimar el rendimiento esperado en datos no vistos. Con el fin de facilitar la corrección, cada decisión queda respaldada por resultados cuantitativos (métricas, tiempos de ejecución y visualizaciones) y se controla la aleatoriedad mediante una semilla fija para garantizar reproducibilidad.

Ha sido realizada por Diego Valladares - 100475849

In [16]:
import numpy as np
import pandas as pd

ava_data = pd.read_pickle ("bank_05.pkl")
comp_data = pd.read_pickle ("bank_competition.pkl")

Empiezo visualizando las primeras filas para hacerme una idea rápida de las variables disponibles, el formato y el tipo de información (numéricas vs categóricas).

In [17]:
ava_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,None,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,None,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


Compruebo el tamaño del conjunto de datos. Esto me permite saber cuántas observaciones tengo para entrenar y cuántas variables voy a preprocesar.

In [18]:
ava_data.shape

(11000, 17)

Como ya conocemos las variables predictoras, dejo una tabla con ellas
### Diccionario de variables

| Campo       | Descripción |
|------------|-------------|
| `age`      | Edad del cliente |
| `job`      | Tipo de trabajo |
| `marital`  | Estado civil |
| `education`| Nivel de formación |
| `default`  | ¿Tiene créditos no devueltos? |
| `balance`  | Saldo medio anual (€) |
| `housing`  | ¿Tiene préstamos hipotecarios? |
| `loan`     | ¿Tiene un préstamo personal? |
| `contact`  | Tipo de comunicación de contacto |
| `day`      | Último día de contacto del mes (numérico) |
| `month`    | Último mes de contacto del año |
| `duration` | Duración del último contacto, en segundos |
| `campaign` | Número de contactos realizados durante esta campaña y para este cliente |
| `pdays`    | Número de días transcurridos desde la última vez que se contactó con el cliente en una campaña anterior (numérico, `-1` significa que no se contactó previamente con el cliente) |
| `previous` | Número de contactos realizados antes de esta campaña y para este cliente |
| `poutcome` | Resultado de la campaña de marketing anterior |
| `deposit`  | ¿El cliente ha suscrito un depósito a plazo fijo? (variable objetivo) |

> Nota: `deposit` es la variable objetivo (clasificación binaria). En el conjunto `bank_competition` no aparece esta columna.

### Estructura y valores no nulos

A continuación utilizo `info()` para obtener un resumen rápido del dataset: número de filas/columnas, tipo de dato de cada variable y, sobre todo, el recuento de valores **no nulos** por columna, lo que permite detectar de forma inmediata si existen variables con valores perdidos.

In [19]:
ava_data.info()

<class 'pandas.DataFrame'>
Index: 11000 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11000 non-null  int64 
 1   job        11000 non-null  object
 2   marital    10468 non-null  object
 3   education  11000 non-null  object
 4   default    11000 non-null  object
 5   balance    11000 non-null  int64 
 6   housing    11000 non-null  object
 7   loan       11000 non-null  object
 8   contact    11000 non-null  object
 9   day        11000 non-null  int64 
 10  month      11000 non-null  object
 11  duration   11000 non-null  int64 
 12  campaign   11000 non-null  int64 
 13  pdays      11000 non-null  int64 
 14  previous   11000 non-null  int64 
 15  poutcome   11000 non-null  object
 16  deposit    11000 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.5+ MB


A partir de `info()` se observa que el dataset contiene **11.000 registros y 17 variables**. En cuanto a calidad de datos, todas las columnas aparecen completas excepto **`marital`**, que tiene **10.468 valores no nulos** (por tanto, **532 valores faltantes**). Esto sugiere que será necesario aplicar una estrategia de imputación para esta variable durante el preprocesamiento.

Para confirmar cómo están representados esos valores faltantes en `marital`, inspecciono sus valores únicos. En particular, quiero comprobar si los “missing” aparecen como `NaN`, como la categoría `"unknown"` o como valores nulos tipo `None`.

In [20]:
ava_data["marital"].unique()

array(['married', None, 'single', 'divorced'], dtype=object)

Aqui podemos observar la cantidad exacta para cada opcion dentro marital

In [21]:
ava_data["marital"].value_counts(dropna=False)

marital
married     5960
single      3292
divorced    1216
None         532
Name: count, dtype: int64

Aqui nos da de nuevo otra forma de ver la cantidad de nulos en este caso como porcentaje

In [22]:
missing = ava_data.isna().mean().sort_values(ascending=False) * 100
missing[missing > 0]

marital    4.836364
dtype: float64

-------------------------------------------------------------------------------------------------------------------------------------------------
### Variable objetivo: balance de clases

Antes de profundizar en las variables, reviso la distribución de `deposit` para comprobar si el problema está desbalanceado, ya que esto condiciona la métrica a optimizar y la forma de evaluar los modelos.

In [23]:
ava_data["deposit"].value_counts()

deposit
no     5780
yes    5220
Name: count, dtype: int64

In [24]:
ava_data["deposit"].value_counts(normalize=True) * 100

deposit
no     52.545455
yes    47.454545
Name: proportion, dtype: float64

Observamos que la variable objetivo está **relativamente balanceada** (`no` ≈ 52.5% y `yes` ≈ 47.5%)

-------
### Tipos de variables

Separo variables numéricas y categóricas para analizar sus distribuciones y preparar el preprocesamiento posterior (escalado para numéricas y codificación para categóricas).

In [25]:
num_cols = ava_data.select_dtypes(include=["int64", "float64"]).columns
cat_cols = ava_data.select_dtypes(include=["object", "category"]).columns

num_cols, cat_cols

(Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object'),
 Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
        'month', 'poutcome', 'deposit'],
       dtype='object'))

Ahora vamos a contar cuantas filas con -1 aparece en pdays

In [26]:
(ava_data["pdays"] == -1).sum()
(ava_data["pdays"] == -1).mean() * 100

np.float64(74.57272727272726)

Vamos a crear una columna nueva donde 1 si se ha contactado y 0 si nunca le han contactado

In [27]:
ava_data["npdays"] = (ava_data["pdays"] != -1).astype(int)
ava_data["pdays"] = ava_data["pdays"].replace(-1, np.nan)

Separacion de X/y, hacer el split train/test (2/3–1/3)

In [28]:
X = ava_data.drop(columns=["deposit"])
y = ava_data["deposit"].map({"no": 0, "yes": 1})

y.value_counts(normalize=True)

deposit
0    0.525455
1    0.474545
Name: proportion, dtype: float64

In [29]:
from sklearn.model_selection import train_test_split

SEED = 100475849
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1/3, random_state=SEED, stratify=y
)

In [30]:
num_cols = X_train.select_dtypes(include=["number"]).columns
cat_cols = X_train.select_dtypes(exclude=["number"]).columns

In [31]:
ava_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,npdays
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,NaN,0,unknown,yes,0
1,56,admin.,None,secondary,no,45,no,no,unknown,5,may,1467,1,NaN,0,unknown,yes,0
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,NaN,0,unknown,yes,0
3,55,services,None,secondary,no,2476,yes,no,unknown,5,may,579,1,NaN,0,unknown,yes,0
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,NaN,0,unknown,yes,0
